In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
from data_handle import *
from gensim_wrapper import *

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def conv_model(n_input, n_output, n_enc_units, n_dec_units):
    root_word_input = Input(shape=(13, 309, 1), name="root_word_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    
    x = Flatten()(x)

    state_h = Dense(n_dec_units, activation='relu')(x)
    
    decoder_inputs = Input(shape=(None, 309), name="target_word_input")
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(309, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs], decoder_outputs)
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model


In [78]:
def embedding_model(embed_size):
    context_word = Input(shape=(embed_size,), name="context_word")
    target_word = Input(shape=(embed_size,), name="target_word")
    embeder = Dense(200, activation='tanh')
    con_emb = embeder(context_word)
    tar_emb = embeder(tar_emb)
    target_word = Dense(embed_size, activation='tanh')(embeding)
    model = Model(context_word, target_word)
    em_model = Model(context_word, embeding)
    return model, em_model

In [4]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()

n_chars = 11 + 2
n_features = len(char2int)
batch_size = 128
embed_size = 128

In [5]:
gen = generate_word_images(vocab, char2int, batch_size)

In [6]:
train, infenc, infdec = conv_model(13, 13, embed_size, embed_size)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [7]:
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, 13, 309, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 13, 309, 16)  160         root_word_input[0][0]            
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 6, 154, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 6, 154, 8)    1160        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [8]:
n_batches = len(vocab) // batch_size
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 2)

Epoch 1/2
2326/2326 [==============================] - 294s 127ms/step - loss: 0.3463 - acc: 0.9510
Epoch 2/2
2326/2326 [==============================] - 283s 122ms/step - loss: 0.0626 - acc: 0.9961


In [9]:
embeddings = []
word2int = {}
i = 0
print(len(vocab))
for word in vocab:
    word2int[word] = len(word2int)
    vec = word2vec(char2int, word, 13).reshape((1, 13, 309, 1))
    emb = infenc.predict(vec)
    embeddings.append(emb)
    if i % 10000 == 0:
        print(i * 100.0 / len(vocab))
    i+=1

297836
0.0
3.3575524785452395
6.715104957090479
10.072657435635719
13.430209914180958
16.787762392726197
20.145314871271438
23.50286734981668
26.860419828361916
30.217972306907157
33.575524785452394
36.93307726399764
40.290629742542876
43.64818222108811
47.00573469963336
50.363287178178595
53.72083965672383
57.07839213526908
60.435944613814314
63.79349709235955
67.15104957090479
70.50860204945003
73.86615452799528
77.22370700654051
80.58125948508575
83.938811963631
87.29636444217623
90.65391692072147
94.01146939926672
97.36902187781195


In [11]:
gensim = GensimWrapper(embed_size, 0, log=True)
embeds = np.stack(embeddings).reshape((-1, 128))
norms = np.linalg.norm(embeds, axis=1, keepdims=True)
embeds_normal = embeds / norms

2018-10-24 16:18:20,852 : INFO : collecting all words and their counts
2018-10-24 16:18:20,853 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-24 16:18:20,912 : INFO : PROGRESS: at sentence #10000, processed 150787 words, keeping 38040 word types
2018-10-24 16:18:21,025 : INFO : PROGRESS: at sentence #20000, processed 309265 words, keeping 61588 word types
2018-10-24 16:18:21,080 : INFO : PROGRESS: at sentence #30000, processed 477795 words, keeping 81867 word types
2018-10-24 16:18:21,164 : INFO : PROGRESS: at sentence #40000, processed 647737 words, keeping 100652 word types
2018-10-24 16:18:21,227 : INFO : PROGRESS: at sentence #50000, processed 805941 words, keeping 116075 word types
2018-10-24 16:18:21,291 : INFO : PROGRESS: at sentence #60000, processed 967897 words, keeping 129675 word types
2018-10-24 16:18:21,365 : INFO : PROGRESS: at sentence #70000, processed 1144759 words, keeping 143321 word types
2018-10-24 16:18:21,417 : INFO : PROGRESS

In [13]:
gensim.set_embeddings(word2int, embeds_normal)
gensim.evaluate()

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
2018-10-24 16:18:49,495 : INFO : semantic word embedding: 0.0% (0/185)
2018-10-24 16:18:49,495 : INFO : total: 0.0% (0/185)
2018-10-24 16:18:50,030 : INFO : syntactic evaluation: 9.8% (13/132)
2018-10-24 16:18:50,031 : INFO : total: 9.8% (13/132)


73.39449541284404


In [16]:
from utils import *

utils = Utils(word2int, embeds_normal)

In [17]:
utils.sorted_sim("ኢትዮጵያን")

[('ኢትዮጵያን', 1.0),
 ('አትዮጵያን', 0.9997844),
 ('ኢትዮጵያና', 0.9996847),
 ('ኢትዮጵያያን', 0.99963176),
 ('ኢትዮጵያንና', 0.9995229),
 ('ኢትዮጵያንን', 0.9994656),
 ('ኤትዮጵያና', 0.99933463),
 ('ኢትዮጵያንም', 0.9992549),
 ('የኢትዮጵያን', 0.99925405),
 ('ኢትዮዮጵያና', 0.9991602)]

In [35]:
def generate(data, embeds, word2int, batch_size, skip_window):
    embed_szie = embeds.shape[1]
    assert batch_size % skip_window == 0
    ci = skip_window  # current_index
    while True:
        batch_inputs = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_labels = np.ndarray(shape=(batch_size, embed_size), dtype=np.float32)
        batch_index = 0
        for batch_index in range(0, batch_size, skip_window * 2):  # fill the batch inputs
            context = data[ci - skip_window:ci + skip_window + 1]
            # remove the target from context words
            target = context.pop(skip_window)
            # context = random.sample(context, skip_window * 2)
            word_index = 0
            for b in range(batch_index, batch_index + skip_window * 2):
                con_vec = embeds[word2int[context[word_index]]]
                target_vec = embeds[word2int[target]]
                batch_inputs[b] = con_vec
                batch_labels[b] = target_vec
                word_index += 1

            ci += 1
        if len(data) - ci - skip_window < batch_size:
            ci = skip_window
        yield batch_labels, batch_inputs


In [86]:
g = generate(words, embeds_normal, word2int, batch_size=120, skip_window=5)

In [87]:
em_train, em_out = embedding_model(embed_size)
adam = keras.optimizers.adam(lr=0.001)
em_train.compile(optimizer=adam, loss='hinge')
em_train.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
context_word (InputLayer)    (None, 128)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 200)               25800     
_________________________________________________________________
dense_39 (Dense)             (None, 128)               25728     
Total params: 51,528
Trainable params: 51,528
Non-trainable params: 0
_________________________________________________________________


In [88]:
n_batches = len(words) // 120
history = em_train.fit_generator(g, steps_per_epoch=n_batches, epochs = 2)

Epoch 1/2
34078/34078 [==============================] - 162s 5ms/step - loss: 0.9478
Epoch 2/2
34078/34078 [==============================] - 165s 5ms/step - loss: 0.9478


In [82]:
semantic = em_out.predict(embeds_normal)
gensim = GensimWrapper(200, 0, log=True)
norms = np.linalg.norm(semantic, axis=1, keepdims=True)
semantic_normal = semantic / norms

2018-10-24 18:16:12,318 : INFO : collecting all words and their counts
2018-10-24 18:16:12,319 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-24 18:16:12,371 : INFO : PROGRESS: at sentence #10000, processed 150787 words, keeping 38040 word types
2018-10-24 18:16:12,420 : INFO : PROGRESS: at sentence #20000, processed 309265 words, keeping 61588 word types
2018-10-24 18:16:12,484 : INFO : PROGRESS: at sentence #30000, processed 477795 words, keeping 81867 word types
2018-10-24 18:16:12,548 : INFO : PROGRESS: at sentence #40000, processed 647737 words, keeping 100652 word types
2018-10-24 18:16:12,611 : INFO : PROGRESS: at sentence #50000, processed 805941 words, keeping 116075 word types
2018-10-24 18:16:12,667 : INFO : PROGRESS: at sentence #60000, processed 967897 words, keeping 129675 word types
2018-10-24 18:16:12,726 : INFO : PROGRESS: at sentence #70000, processed 1144759 words, keeping 143321 word types
2018-10-24 18:16:12,778 : INFO : PROGRESS

In [85]:
gensim.set_embeddings(word2int, semantic_normal)
gensim.evaluate()

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
2018-10-24 18:16:46,287 : INFO : semantic word embedding: 0.0% (0/185)
2018-10-24 18:16:46,288 : INFO : total: 0.0% (0/185)
2018-10-24 18:16:46,842 : INFO : syntactic evaluation: 3.0% (4/132)
2018-10-24 18:16:46,842 : INFO : total: 3.0% (4/132)


72.4770642201835
